In [1]:
import torch
from tqdm import tqdm
from model import seedformer_dim128
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset,DataLoader
import open3d as o3d
import os
from pytorch3d.loss import chamfer_distance
import gc
torch.cuda.empty_cache()
gc.collect()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


0

### Dataset and Dataloader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

In [2]:
class RacingDataset(Dataset):
    def __init__(self,root_dir):
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)
        self.filter_file_list = self.filter_list()

    def __len__(self):
        return len(self.filter_file_list)

    def __getitem__(self,index):
        pcd_path = os.path.join(self.root_dir,self.filter_file_list[index])
        pcd = o3d.io.read_point_cloud(pcd_path)

        points = torch.tensor(pcd.points, dtype=torch.float32)

        return points,pcd_path

    def filter_list(self):
        '''
        Filter the inputs so that only pcds with more than 50 points are included in the training
        :return:
        '''
        filtered_list=[]
        for filename in self.file_list:
            pcd = o3d.io.read_point_cloud(os.path.join(self.root_dir,filename))
            points = torch.tensor(pcd.points, dtype=torch.float32)
            if len(points)>=500:
                filtered_list.append(filename)
        return filtered_list
   


In [12]:
 def collate_fn(batch):
        points_batch = [item[0] for item in batch]
        padded_points = pad_sequence(points_batch, batch_first=True, padding_value=0)
    
        paths = [item[1] for item in batch]
    
        return padded_points, paths
dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=8, collate_fn=collate_fn)
print(len(dataloader))
print(dataloader.dataset)

143


In [13]:
#from numba import cuda
# 
#cuda.select_device(0) # choosing second GPU 
#cuda.close()

In [16]:
#torch.manual_seed(42)
#del model
model=seedformer_dim128(up_factors=[1, 2, 3])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
epochs=5
optimizer=torch.optim.Adam(model.parameters(),lr=0.01, weight_decay=5e-4)
for epoch in range(epochs):
    running_loss=0
    # Wrap the DataLoader with tqdm to track progress
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f'Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
        for i, data in pbar:
            inputs, labels = data
            inputs = inputs.to(device)  # Move data to GPU if available

            optimizer.zero_grad()
            outputs = model(inputs)
            losses = []

            for input_pc, output_pc in zip(inputs, outputs):
                # Calculate Chamfer Distance loss using pytorch3d.loss.chamfer_distance
                loss, _ = chamfer_distance(input_pc.unsqueeze(0), output_pc.unsqueeze(0))
                losses.append(loss)

            loss = torch.mean(torch.stack(losses))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1))  # Update tqdm progress bar with the current loss

    print(f'Epoch {epoch+1} Loss: {running_loss / len(dataloader)}')  # Log the epoch loss

print('Finished Training')


Epoch 1/5: 100%|██████████| 572/572 [00:42<00:00, 13.52batch/s, loss=1.12e+23]

Epoch 1 Loss: 1.1227867836764279e+23



Epoch 2/5: 100%|██████████| 572/572 [00:42<00:00, 13.57batch/s, loss=6.89e+22]

Epoch 2 Loss: 6.887807833706316e+22



Epoch 3/5: 100%|██████████| 572/572 [00:42<00:00, 13.47batch/s, loss=2.69e+22]

Epoch 3 Loss: 2.6904143257812215e+22



Epoch 4/5: 100%|██████████| 572/572 [00:43<00:00, 13.21batch/s, loss=1.65e+22]

Epoch 4 Loss: 1.6504053970516025e+22



Epoch 5/5: 100%|██████████| 572/572 [00:43<00:00, 13.13batch/s, loss=1.06e+22]

Epoch 5 Loss: 1.0640143194541649e+22
Finished Training


In [17]:

from pytorch3d.loss import chamfer_distance

# Define your model and dataset class

# Load simulation dataset
simulation_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/sim")
simulation_dataloader = DataLoader(simulation_dataset, batch_size=1, shuffle=False, num_workers=8)

# Load real dataset
real_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
real_dataloader = DataLoader(real_dataset, batch_size=1, shuffle=False, num_workers=8)

# Select a subset of samples
num_samples = 3

# Apply the model and visualize differences
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

for dataset, dataloader, name in [(simulation_dataset, simulation_dataloader, "Simulation"), 
                                  (real_dataset, real_dataloader, "Real")]:
    for i, data in enumerate(tqdm(dataloader, desc=f'Processing {name} dataset', unit='point cloud')):
        if i >= num_samples:
            break
        
        inputs, paths = data
        inputs = inputs.to(device)

        with torch.no_grad():
            outputs = model(inputs)

        original_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(inputs.squeeze().cpu().numpy()))
        original_pc.paint_uniform_color([1, 0, 0])  # Paint original point cloud in red

        reconstructed_pc = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(outputs.squeeze().cpu().numpy()))
        reconstructed_pc.paint_uniform_color([0, 1, 0])  # Paint reconstructed point cloud in green

        o3d.visualization.draw_geometries([original_pc, reconstructed_pc], window_name=f"{name} - Sample {i+1}")
model=None

Processing Real dataset:   1%|          | 3/572 [00:34<1:47:51, 11.37s/point cloud]
